# Combining Datasets: Merge and Join

Одним очень важным преимуществом Pandas является его высокая производительность, операции соединения и объединения в памяти. Главным интерфейсом является функция `pd.merge` и мы увидим как она работает.

Для удобства переопределим функциональность `display()`:

In [2]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(
            self.template.format(a, eval(a)._repr_html_()) for a in self.args
        )
    
    def __repr__(self):
        return '\n\n'.join(
            a + '\n' + repr(eval(a)) for a in self.args
        )

## Реляционная алгебра

Поведение `pd.merge()` является подмножеством того, что известно как _реляционная алгебра_, что является формальным набором правил для манипуляций с реляционными даными, и формирует концептуальную базу операций доступных в большинстве баз данных.

Pandas реализует несколько из этих базовых строительных блоков в функции `pd.merge()` и связан с методом `join()` объектов `Series` и `DataFrame`.

## Категории соединений

`pd.merge()` реализует несколько операций соединения: _один-к-одному_, _многие-к-одному_ и _многие-ко-многим_. Все три типа доступные через вызов `pd.merge()`, тип соединения зависит от формы входных данных.

### Соединения один-к-одному

_Один-к-одному_, возможно, является простейшим типом соединения , который во многом очень похож на объединение рассмотренное в [Combining Datasets: Concat & Append](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html). В качесте конкретного примера рассмотрим следующие два объекта `DataFrame`, которые содержат информацию по нескольким работникам в компании:

In [3]:
df1 = pd.DataFrame(
    {'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
     'group': ['Accounting', 'Engineering', 'Engineering', 'HR']}
)
df2 = pd.DataFrame(
    {'employee': ['Lisa', 'Bob', 'Jake', 'Sue'], 'hire_date': [2004, 2008, 2012, 2014]}
)
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


Для объединения этих данных в один `DataFrame` мы можем использовать функцию `pd.merge()`:

In [4]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


`pd.merge()` определяет, что каждый объект `DataFrame` содержит колонку "employee" и автоматически соединяет наборы данных используя эту колонку как ключ. Результат соединения - новый объект `DataFrame` который содержит информацию из двух источников. Обратите внимение, что `merge` в общем отбрасывает индексы, кроме специальных случаев соединений по индексу (см. `left_index` и `right_index` рассмотренные далее).

### Соединения многие-к-одному

Соединения типа _многие-к-одному_ используются в случаях когда одна или две ключевых колонок содержат повторяющиеся данные. Для случая многие-к-одному, результирующий `DataFrame` будет содержать эти дубликаты.

In [5]:
df4 = pd.DataFrame(
    {'group': ['Accounting', 'Engineering', 'HR', 'R&D'], 'supervisor': ['Carly', 'Guido', 'Steve', 'Bill']}
)
merge34 = pd.merge(df3, df4)
display('df3', 'df4', 'merge34')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve
3          R&D       Bill

merge34
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

Результирующий `DataFrame` содержит дополнительный столбец "supervisor", в котором информация повторяется в одной или более локаций.

Также в `df4` содержится дополнительная группа `R&D` с руководителем `Bill`, но в итоговый результат она не попала, так как в `df3` нет соответствующего работника из `R&D`.

### Соединения многие-ко-многим

Если ключевая колонка в обоих левом и правом массивах содержит дубликаты, тогда результат будет соединение _многие-ко-многим_. Например, мы имеет `DataFrame` показывающий один или несколько навыков ассоциированных с конкретной группой. Выполняя соединение _многие-ко-многим_ мы можем найти навыки ассоциированные с каждым работником:

In [6]:
df5 = pd.DataFrame(
    {
        'group': ['Accounting', 'Accounting', 'Engineering', 'Engineering', 'HR', 'HR'],
        'skills': ['math', 'spreadsheets', 'coding', 'linux', 'spreadsheets', 'organization']
    }
)
merge15 = pd.merge(df1, df5)
display('df1', 'df5', 'merge15')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


## Описание ключа для соединения

Мы уже рассмотрели поведение по умолчанию для `pd.merge()`: найти одну или более колонок с одинаковыми именами в двух разных входных данных, использовать эту(и) колонку(и) в качестве ключа. Однако имена колонок не всегда соответствуют и `pd.merge()` предлагает множество опций для работы с такими данными.

### Ключевое слово `on`

Вы можете просто явно указать имя ключевой колонки, используя ключевое слово `on`, которое принимает имя колонки или список имён колонок. Эта опция работает только если в обеих левом и правом `DataFrame` имеются указанные имена колонок.

In [7]:
merge12 = pd.merge(df1, df2, on='employee')
display('df1', 'df2', 'merge12')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

merge12
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

### Ключевые слова `left_on` и `right_on`

Иногда вам может быть необходимо соединить два набора данных с различными именами колонок; например, у нас может быть набор данных в котором имя работника находится в колонке "name", а не "employee". В этом случае мы можем использовать ключевые слова `left_on` и `right_on` для указания двух имён колонок:

In [8]:
df3 = pd.DataFrame(
    {'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 'salary': [70000, 80000, 120000, 90000]}
)
merge13 = pd.merge(df1, df3, left_on="employee", right_on="name")
display('df1', 'df3', 'merge13')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

merge13
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

Результат содержит избыточную колонку, которую мы можем удалить при необходимости:

In [9]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


### Ключевые слова `left_index` и `right_index`

Иногда, вместо соединения по колонкам, вам необходимо объединить по индексу.

In [10]:
df1a = df1.set_index("employee")
df2a = df2.set_index("employee")
display("df1a", "df2a")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Вы можете использовать индекс в качестве ключа для соединения с помощью указания флагов `left_index` и/или `right_index` в `pd.merge()`:

In [11]:
merge1a2a = pd.merge(df1a, df2a, left_index=True, right_index=True)
display('df1a', 'df2a', 'merge1a2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Для удобства, класс `DataFrame` реализует метод `join()`, который, по умолчанию, выполняет соединение по индексам:

In [12]:
merge1a2a = df1a.join(df2a)
display('df1a', 'df2a', 'merge1a2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


Если вы хотите совместить индексы и колонки, мы можете скомбинировать `left_index` с `right_on` или `left_on` с `right_index`:

In [13]:
merge1a3 = pd.merge(df1a, df3, left_index=True, right_on='name')
display('df1a', 'df3', 'merge1a3')

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

merge1a3
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

Все эти опции также работают с множественными индексами и/иди множественными колонками, интерфейс достаточно интуитивен. Для получения большей информации обратитесь к секции [Merge, Join and Concatenate](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) в документации Pandas.

## Указание арифметики множеств при выполнении соединения

В примерах выше мы не рассмотрели одно важное соображение при выполнении соединений: тип операции из арифметики множеств, используемой в соединении. Это может быть важно в ситуации когда значение присутствует в одной ключевой клонке и отсутствует в другой. Например:

In [14]:
df6 = pd.DataFrame(
    {'name': ['Peter', 'Paul', 'Mary'], 'food': ['fish', 'beans', 'bread']},
    columns=['name', 'food']
)
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'], 'drink': ['wine', 'beer']}, columns=['name', 'drink'])
merge67 = pd.merge(df6, df7)
display('df6', 'df7', 'merge67')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

merge67
   name   food drink
0  Mary  bread  wine

В примере выше мы выполнини соединение и оказалось, что в колонке "name" только одно общее значение: Mary. По умолчанию, результат содержит _пересечение_ двух входных множеств, это то, что называется внутренним соединением (_inner join_). Мы можем явно задать тип, с помощью ключевого слова `how`, значение по умолчанию для которого: `inner`:

In [15]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Другие опции для ключевого слова `how`: `outer`, `left` и `right`. Внешнее соединение _outer join_, возвращает соединение по объенинённому множеству входных колонок и заполняет отсутствующие значения с `NA`:

In [16]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

Левое соединение (_left join_) и правое соединение (_right join_), работают по значениям слева и справа соответственно. Например:

In [17]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

Выходные ряды теперь содержат записи из левого входа. Использование `how='right'` работает аналогично.

Все эти опции могут быть применены напрямую к любому из предыдущих типов соединений.

## Пересекающиеся имена колонок: ключевое слово `suffixes`

В какой-то момент вам может понадобится два `DataFrame` содержащие конфликтующие имена. Например:

In [20]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'], 'rank': [3, 1, 4, 2]})
merge89 = pd.merge(df8, df9, on="name")
display('df8', 'df9', 'merge89')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

merge89
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

Из-за того, что наборы данных содержат колонки с конфликтующими именами, соединение, чтобы сделать имена колонок уникальными, автоматически добавляет суффиксы `_x` и `_y`. Если эти новые имена не подходят, то существует возможность задать пользовательский суффикс с помощью ключевого слова `suffixes`:

In [21]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

## Пример: данные штатов США

Операции объединения и соединения наиболее часто используются когда надо объединить данные из разных источников. Далее мы рассмотрим пример данных о штатах США и их населения. Сами данные расположены тут http://github.com/jakevdp/data-USstates/

In [23]:
# Following are shell commands to download the data
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

Давайте загрузим эти данные с помощью функции Pandas `read_csv()`:

In [27]:
pop = pd.read_csv('notebooks/data/state-population.csv')
areas = pd.read_csv('notebooks/data/state-areas.csv')
abbrevs = pd.read_csv('notebooks/data/state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

Имея эти данные давайте попробуем получить относительно простой результат: рейтинг штатов США и территорий по уровню плотности населения на 2010 год. У нас имеются все данные, но нам надо будет объединить наборы данных, чтобы получить результат.

Мы начнем с объединения `многие-к-одному`, которое даст нам название штата для набора данных по населению. Нам надо объединить на основе колонки `state/region` из `pop` и колонки `abbreviation` из `abbrevs`. Мы будем использовать `how='outer'` чтобы гарантировать, что никакие данные не пропадут в случае наличия каких-то несовпадений.

In [31]:
merged = pd.merge(pop, abbrevs, how='outer', left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', axis=1) # drop duplicate info
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Давайте найдём все несоответсвия, которые мы можем найти с помощью поиска `null` значений:

In [33]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Что-то в колонке `population` является пустым значением, давайте их найдём.

In [36]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


Похоже, что все пустые значения в колонке `population` относятся к Пуэрто-Рико к периоду до 2000 года, скорее всего они вообще недоступны в оригинальном источнике.

Что более важно, так это то, что некоторые записи в колонке `state` тоже пустые, что означает, что в наборе `abbrevs` не оказалось соответствующих значений. Давайте найдём какие регионы не имеют соответствий:

In [38]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

Мы можем сделать вывод, что данные о населении включают данные для Пуэрто-Рико (PR) и всем США (USA), в то время как эти данные не имеют значений в таблице сокращений. Мы можем это быстро починить путем заполнения соответствующих значений:

In [39]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

Теперь мы можем объединить результаты с данными по площадям (`areas`) используя аналогичный подход. Мы будем использовать колонку `state` для объединения:

In [40]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Давайте снова проверим на наличие пустых значений:

In [42]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

Пустые значения присутствуют в колонке `area (sq. mi)`, мы можем узнать какие регионы были проигнорированы:

In [43]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

Мы видим, что набор данных не содержит данные о площади США в целом. Мы могли бы вставить подходящее значение, но, в данном случае, мы просто удалим пустое значение, посколько плотность населения во всех США нам сейчас неинтересна:

In [44]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


Теперь у нас есть все данные. Для начала давайте выберем порцию данных относящуюся к 2010 году и общему количеству населения. Мы используем фукнцию `query()`:

In [45]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Теперь давайте рассчитаем плотность населения и выведем ее в отсортированном виде. Мы начнём с ре-индексирования наших данных по штату и потом рассчитаем результат:

In [46]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [49]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

Мы получили, отсортированный по плотности населения на одну квадратную милю, список штатов США, а также Вашингтон (Округ Колумбия) и Пуэрто-Рико.